# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra
using PseudoPotentialData

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.largecore.gth")
# Helium at the center of the box
atoms     = [ElementPsp(:He, pseudopotentials)]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model  = model_DFT(lattice, atoms, positions;
                   functionals=LDA(), symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis; tol)
μref   = dipole(basis, scfres.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770326766320                   -0.53    9.0    190ms
  2   -2.771678441810       -2.87       -1.31    1.0   95.9ms
  3   -2.771713615975       -4.45       -2.63    1.0    102ms
  4   -2.771714712765       -5.96       -3.99    2.0    118ms
  5   -2.771714715186       -8.62       -4.64    2.0    122ms
  6   -2.771714715246      -10.22       -5.45    1.0    109ms
  7   -2.771714715250      -11.47       -6.68    2.0    125ms
  8   -2.771714715250   +  -14.12       -7.37    1.0    106ms
  9   -2.771714715250   +  -15.05       -8.32    2.0    127ms


-0.00013457422863293842

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770497567928                   -0.53    8.0    203ms
  2   -2.771772175687       -2.89       -1.32    1.0    103ms
  3   -2.771801575237       -4.53       -2.53    1.0    119ms
  4   -2.771802025029       -6.35       -3.46    1.0    116ms
  5   -2.771802073175       -7.32       -3.97    2.0    207ms
  6   -2.771802074443       -8.90       -5.41    1.0    869ms
  7   -2.771802074475      -10.50       -5.45    2.0    123ms
  8   -2.771802074476      -11.81       -6.51    1.0    109ms
  9   -2.771802074476      -13.18       -6.72    2.0    161ms
 10   -2.771802074476   +  -13.97       -7.38    1.0    133ms
 11   -2.771802074476      -13.99       -8.27    2.0    126ms


0.01761222107954828

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457422863293842
Displaced dipole:  0.01761222107954828
Polarizability :   1.7746795308181216


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response

Now we use linear response (also known as density-functional perturbation theory)
to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field).

In [5]:
# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

54×54×54×1 Array{Float64, 4}:
[:, :, 1, 1] =
  5.0       5.0       5.0       5.0      …   5.0       5.0       5.0
  4.81481   4.81481   4.81481   4.81481      4.81481   4.81481   4.81481
  4.62963   4.62963   4.62963   4.62963      4.62963   4.62963   4.62963
  4.44444   4.44444   4.44444   4.44444      4.44444   4.44444   4.44444
  4.25926   4.25926   4.25926   4.25926      4.25926   4.25926   4.25926
  4.07407   4.07407   4.07407   4.07407  …   4.07407   4.07407   4.07407
  3.88889   3.88889   3.88889   3.88889      3.88889   3.88889   3.88889
  3.7037    3.7037    3.7037    3.7037       3.7037    3.7037    3.7037
  3.51852   3.51852   3.51852   3.51852      3.51852   3.51852   3.51852
  3.33333   3.33333   3.33333   3.33333      3.33333   3.33333   3.33333
  ⋮                                      ⋱                      
 -3.33333  -3.33333  -3.33333  -3.33333  …  -3.33333  -3.33333  -3.33333
 -3.51852  -3.51852  -3.51852  -3.51852     -3.51852  -3.51852  -3.51852
 -3.7037   -3.7037 

Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

First we do this using the `DFTK.solve_ΩplusK_split`
function provided by DFTK,
which uses an adaptive Krylov subspace algorithm [^HS2025]:

[^HS2025]:
    M. F. Herbst and B. Sun.
    *Efficient Krylov methods for linear response in plane-wave electronic structure calculations.*
    [arXiv 2505.02319](http://arxiv.org/abs/2505.02319)

In [6]:
# Multiply δVext times the Bloch waves, then solve the Dyson equation:
δVψ = DFTK.multiply_ψ_by_blochwave(scfres.basis, scfres.ψ, δVext)
res = DFTK.solve_ΩplusK_split(scfres, δVψ; verbose=true)

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.0  71.8ms  Non-interacting
   1        0       1       -0.60     10.0   1.17s  
   2        0       2       -2.42      7.0   130ms  
   3        0       3       -3.54      5.0   105ms  
   4        0       4       -5.33      4.0   101ms  
   5        0       5       -7.38      1.0  87.0ms  
   6        0       6      -10.16      1.0  87.3ms  
   7        1       1       -7.39     13.0   236ms  Restart
   8        1       2       -8.98      1.0  89.8ms  
                                      13.0   138ms  Final orbitals


(δψ = Matrix{ComplexF64}[[0.002907788270514669 + 0.0015516758802247292im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.1479478147681393 - 0.27173082323947956im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; … ; 0.019478973687578224 - 0.03615129913864817im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; -0.03783112630678848 + 0.07155276231979216im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im]], δρ = [-3.6206956481647244e-7 -2.5020161555239363e-7 … -4.880645688612678e-8 -2.502015929364735e-7; -6.293618250026305e-7 -4.793037663300426e-7 … -1.2077849469099973e-7 -4.793037555541758e-7; … ; 1.3626262848631e-7 1.1356231657388693e-7 … 4.711994265489009e-8 1.1356232646456696e-7; 1.3305471972904913e-7 1.3632636520810422e-7 … 5.2006866882094983e-8 1.3632638689367047e-7;;; -2.502015585213608e-7 -1.740491235867452e-7 … -3.556495714152604e-8 -1.740491071443778e-7; -4.79303727344353e-7 -3.679732751567065e-7 … -1.1202899456028139e-7 -3.6797326805786524e-7; … ; 1.135623306502646e-7 1.1211101296897998e-7 … 1.098068818331781e-7 1.1211102129

From the result of `solve_ΩplusK_split` we can easily compute the polarisabilities:

In [7]:
println("Non-interacting polarizability: $(dipole(basis, res.δρ0))")
println("Interacting polarizability:     $(dipole(basis, res.δρ))")

Non-interacting polarizability: 1.9257125370315598
Interacting polarizability:     1.7736548663770435


As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.

### Manual solution of the Dyson equations
To see what goes on under the hood, we also show how to manually solve the
Dyson equation using KrylovKit:

In [8]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; scfres.ρ)
    χ0δV = apply_χ0(scfres, δV).δρ
    δρ - χ0δV
end

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(scfres, δVext).δρ

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve starts with norm of residual = 4.19e+00
[ Info: GMRES linsolve in iteration 1; step 1: normres = 2.49e-01
[ Info: GMRES linsolve in iteration 1; step 2: normres = 3.77e-03
[ Info: GMRES linsolve in iteration 1; step 3: normres = 2.85e-04
[ Info: GMRES linsolve in iteration 1; step 4: normres = 4.69e-06
[ Info: GMRES linsolve in iteration 1; step 5: normres = 1.09e-08
[ Info: GMRES linsolve in iteration 1; step 6: normres = 6.27e-11
[ Info: GMRES linsolve in iteration 1; step 7: normres = 6.58e-10
[ Info: GMRES linsolve in iteration 2; step 1: normres = 6.96e-11
[ Info: GMRES linsolve in iteration 2; step 2: normres = 4.62e-12
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ * norm of residual = 2.79e-13
└ * number of operations = 12
Non-interacting polarizability: 1.9257125370314796
Interacting polarizability:     1.7736548642460341


We obtain the identical result to above.